In [ ]:
sentences = ['bob likes sheep', 'alice is fast', 'cs685 is fun', 'i love lamp']

given first 2 words of sentence, try to predict third, using fixed window nlm

start by tokenizing
convert words to indices

In [2]:
vocab = {} # map from word type to index
inputs = [] # stores an indexified version of rach sentence
for sent in sentences:
    sent_idexs = []
    sent = sent.split() #tokenize w/ whitespace
    for w in sent:
        if w not in vocab:
            vocab[w] = len(vocab) # add a new type to the vocab
        sent_idexs.append(vocab[w])
    inputs.append(sent_idexs)

print(vocab)
print(inputs)

{'bob': 0, 'likes': 1, 'sheep': 2, 'alice': 3, 'is': 4, 'fast': 5, 'cs685': 6, 'fun': 7, 'i': 8, 'love': 9, 'lamp': 10}
[[0, 1, 2], [3, 4, 5], [6, 4, 7], [8, 9, 10]]


In [3]:
import torch

# two things: 1. convert to LongTensors 2. Define inputs/outputs

prefixes = torch.LongTensor([sent[:-1] for sent in inputs ])
labels = torch.LongTensor([sent[-1] for sent in inputs])

print(prefixes, labels)

tensor([[0, 1],
        [3, 4],
        [6, 4],
        [8, 9]]) tensor([ 2,  5,  7, 10])


In [8]:
# Netowrk

import torch.nn as nn

class NLM(nn.Module):
     # two things to do
     # 1. init function (initializes all the params of the network)
     # 2. forward function (defines forward propogation computation)

    def __init__(self, d_embedding, d_hidden, window_size, len_vocab):
        super(NLM, self).__init__() # init the base Module Class
        self.d_emb = d_embedding
        self.embeddings = nn.Embedding(len_vocab, d_embedding)

        # concatenated embeddings -> hidden
        self.W_hid = nn.Linear(d_embedding*window_size, d_hidden)

        # hidden -> output probability distribution over vocab

        self.W_out = nn.Linear(d_hidden, len_vocab)

    def forward(self, input): # each input will be a batch of prefixes
        batch_size, window_size = input.size()
        embs = self.embeddings(input) # 4x2x5
        # print('emb ', embs.size())
        # next concatenate the prefix embeddings together
        concat_embs = embs.view(batch_size, window_size* self.d_emb) # 4 x 10
        # print('conactenated embs ', concat_embs.size())

        #project to hidden space

        hidden = self.W_hid(concat_embs) # 4 x 12

        # print(hidden.size())

        # project to vocabulary space

        out = self.W_out(hidden)

        print('output ', out.size())

        return out # return logits, unnormalized

        # probs = nn.functional.softmax(out, dim=1)

        #print(probs.size())


network = NLM(d_embedding=5, d_hidden=12, window_size=2, len_vocab=len(vocab))
print('labels', labels)
network(prefixes)
        

num_epochs = 50 # num passes through training data
learning_rate = 0.1 #can modify
loss_fn = nn.CrossEntropyLoss() 

# adam, adamax
optimizer = torch.optim.SGD(params=network.parameters(), lr = learning_rate)

# training loop

for i in range(num_epochs):
    logits = network(prefixes)

    print('logits, labels', logits.shape, labels.shape)

    loss = loss_fn(logits,labels)


    #compute gradient (partial derivativ w.r.t loss)

    loss.backward()

    # update params using gradient descent
    optimizer.step()

    #zero gradient for next epoch
    optimizer.zero_grad()

    print('epoch %d ,  loss %0.2f' %(i,loss) )







labels tensor([ 2,  5,  7, 10])
output  torch.Size([4, 11])
output  torch.Size([4, 11])
logits, labels torch.Size([4, 11]) torch.Size([4])
epoch 0 ,  loss 2.59
output  torch.Size([4, 11])
logits, labels torch.Size([4, 11]) torch.Size([4])
epoch 1 ,  loss 2.27
output  torch.Size([4, 11])
logits, labels torch.Size([4, 11]) torch.Size([4])
epoch 2 ,  loss 1.98
output  torch.Size([4, 11])
logits, labels torch.Size([4, 11]) torch.Size([4])
epoch 3 ,  loss 1.72
output  torch.Size([4, 11])
logits, labels torch.Size([4, 11]) torch.Size([4])
epoch 4 ,  loss 1.47
output  torch.Size([4, 11])
logits, labels torch.Size([4, 11]) torch.Size([4])
epoch 5 ,  loss 1.25
output  torch.Size([4, 11])
logits, labels torch.Size([4, 11]) torch.Size([4])
epoch 6 ,  loss 1.04
output  torch.Size([4, 11])
logits, labels torch.Size([4, 11]) torch.Size([4])
epoch 7 ,  loss 0.86
output  torch.Size([4, 11])
logits, labels torch.Size([4, 11]) torch.Size([4])
epoch 8 ,  loss 0.71
output  torch.Size([4, 11])
logits, labe

In [ ]:
# test

